# Personal Assistant and Chatbot

Here I build a personal Alexa/Siri-type assistance named Charlie (after our Shih Tzu). Charlie responds to verbal commands that include the word "Charlie". He can play songs and movies. Charlie is also connected to a chatbot module (chatterbot) and can have basic conversations. 

In [7]:
import speech_recognition as sr
import pyttsx3
import pywhatkit
import datetime
import wikipedia
import webbrowser
import requests
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
import re

In [ ]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.trainers import ChatterBotCorpusTrainer

In [9]:
# create the chatbot
bot = ChatBot(name = 'PyBot', read_only = True, logic_adaptors = ['chatterbot.logic.MathematicalEvaluation','chatterbot.logic.BestMatch'])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...


In [10]:
# train the chatbot using the standard English corpus
trainer = ChatterBotCorpusTrainer(bot)
trainer.train('chatterbot.corpus.english')

Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


With the chatbot built now, I write a chat function When this is triggered, Charlie will act as a chatbot, listening for your lines and saying his own lines.

In [14]:
def chat():
    reply = 'Sure. What do you want to talk about?'
    engine.say(reply)
    engine.runAndWait()
    
    while True:
        try:
            with sr.Microphone() as source:
                voice = listener.listen(source,timeout=6,phrase_time_limit=6)
                _input = listener.recognize_google(voice)
                _input = _input.lower()
                if 'that\'s enough' in _input:
                    engine.say('thank you for the chat')
                    engine.runAndWait()
                    break
                print(_input)
                reply = bot.get_response(_input)
                engine.say(reply)
                engine.runAndWait()

        except:
            pass

I also create a search function that will enable Charlie to search Google.

In [15]:
def search_google(question):
    question = question.strip().replace(' ','+')
    url = 'https://www.google.com/search?q=' + question
    request = urllib.request.Request(url)
    request.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
    raw_response = urllib.request.urlopen(request).read()
    html = raw_response.decode("utf-8")
    soup = BeautifulSoup(html, 'html.parser')
    divs = soup.select("#search div.g")
    
    # get the top answer (the web summary)
    answer = divs[0].get_text()[29:]
                    
    #remove urls and clean up answer
    answer = re.sub(r'http\S+', '', answer)
    answer = answer.split('›')
    answer = answer[np.argmax([len(ans) for ans in answer])]
    
    return answer

Finally, I create the engine using the text to speech module and write a loop that will keep listening for commands until the users tells Charlie "good-bye". In the loop, Charlie can play songs, movies, open the chatbot, search wikipedia for information, tell the time, search google and summarize the top result, and respond to some preset lines (e.g., "Thank you").

In [ ]:
listener = sr.Recognizer()
engine = pyttsx3.init()
engine.say(' I am charlie. What can I do for you?')
engine.runAndWait()

while True:
    try:
        with sr.Microphone() as source:
            print('Listening...')
            #sr.adjust_for_ambient_noise(source)
            voice = listener.listen(source,timeout=6,phrase_time_limit=6)
            command = listener.recognize_google(voice)
            command = command.lower()
            if 'charlie' in command:
                command = command.replace('charlie', '')
                print(command)
                if 'play' in command:
                    song = command.replace('play', '')
                    engine.say('playing' + song)
                    engine.runAndWait()
                    pywhatkit.playonyt(song)
                    print(song)
                elif 'i want to watch' in command:
                    movie = command.replace('i want to watch', '')
                    engine.say('Playing' + movie)
                    engine.runAndWait()
                    movie = movie.strip()
                    movie = movie.replace(' ', '-')
                    webbrowser.open('https://membed.net/videos/' + movie)
                elif 'let\'s chat' in command:
                    chat()    
                elif 'tell me about' in command:
                    thing = command.replace('tell me about', '')
                    info = wikipedia.summary(thing, 2)
                    print(info)
                    engine.say(info)
                    engine.runAndWait()
                elif 'time' in command:
                    time = datetime.datetime.now().strftime('%I: %M')
                    if time[0] == 0:
                        time = time[1:]
                    print(time)
                    engine.say(time)
                    engine.runAndWait()
                elif (
                    command.strip()[0:2] == 'is' 
                    or command.strip()[0:3] == 'are' 
                    or command.strip()[0:3] == 'how' 
                    or command.strip()[0:4] == 'what' 
                    or command.strip()[0:3] == 'why' 
                    or command.strip()[0:3] == 'who'
                ):
                    question = command.strip().replace(' ','+')
                    answer = search_google(question)
                    webbrowser.open('https://www.google.com/search?q=' + question)
                    engine.say(answer)
                    engine.runAndWait()  
                    
                elif 'search for' in command:
                    item = command.replace('search for', '')
                    item = item.replace(' ','+')
                    webbrowser.open('https://www.google.com/search?q=' + item)
                elif 'who are you' in command:
                    engine.say('I am your personal Assistant Charlie')
                    engine.runAndWait()
                elif 'what can you do for me' in command:
                    engine.say('I can play songs and movies and search for information online.')
                    engine.runAndWait()
                elif 'thank you' in command:
                    engine.say('You are welcome')
                    engine.runAndWait()
                elif 'goodbye' in command:
                    engine.say('Goodbye')
                    engine.runAndWait()
                    break
                else:
                    engine.say('I did not understand, please say it again')      
                    engine.runAndWait()

    except:
        pass